In [ ]:
import metadynminer as mm
print(f"Loaded metadynminer version {mm.__version__}.")

In [ ]:
# uncomment the next line to enable %matplotlib widget for interactivity, if you want:
# %matplotlib widget

In [ ]:
# load your HILLS file 
# if the name of your HILLS file is not  the default "HILLS", specify it to the name keyword
# if you don't have your own HILLS file, you can try using name="acealanme", our alanin dipeptide HILLS file used for testing. 
#
hills = mm.Hills(name="HILLS")
# 
# Metadynminer should automatically detect which your CVs are periodic, however, if it fails to detect periodicity correctly, 
# you should specify the periodicity of your CVs as a python list 
# (for example, if you have two periodic CVs, set periodic = [True, True]) like here: 
# 
# hills = mm.Hills(name="HILLS", periodic=[True, True])

In [ ]:
# Calculate the free energy surface from hills file
# decrease the resolution, if you dont need high resolution FES and you find the calculation too slow
# 
# You have two options here: 
# If you use original=False, the FES will be calculted by our Hillsum algorithm - only one Gaussian hill will be precalculated 
# and then moved and scaled and added to the FES. This is less precise, but much faster than calculating each Gaussian explicitly. 
# This is ideal for casual visualisation purposes. 
# 
# If you use original=True, each Gaussian hill will be calculated explicitly, which is more computationally demanding, but also exact. 
# This method is giving the same results as the Plumed's own sum_hills function (tested with Plumed v2.8.0)
# 
fes = mm.Fes(hills, original=False, resolution=256)

In [ ]:
# Identify the local minima
# 
# You have two options: 
#
# If you set the keyword precise=True, the local minima will be identified by an algorithm which: 
# 1. finds all local minima, even very shallow and probably unimportant minima, 
# 2. each point on the FES will be assigned to the minimum the system would most likely go to, 
#    if it only follows the gradient of free energy, and 
# 3. free energy value of minima will be calculated from each point on FES assigned to the respective minima. 
#    This results in more precise free energy values, as it accounts for the width of the minimum as well. 
#    For this calculation the unit of free energy (energy_unit="kJ/mol" or "kcal/mol") and 
#    the thermodynamical temperature (temp) of the simulation must be supplied. 
#    This algorithm doesn't use the nbins keyword. 
# example: 
# minima = mm.Minima(fes, precise=True, temp=300.0, energy_unit="kJ/mol")
# 
# If you set precise = False, the method will use the original algorithm from the metadynminer package for R. 
# In this algorithm the FES is first divided to number of bins (can be set with option nbins, default is 8)
# and the absolute minima is found for each bin. Then the algorithm checks 
# if this point is really a local minimum by comparing to the surrounding points of FES.
# This algorithm only accounts for the depth of each minima, which is less precise, but usually sufficient. 
# 
# In some cases, this algorithm is the prefered one, 
# because on some free energy landscapes the total number of local free energy minima can reach tens of thousands, 
# which makes the calculation using precise algorithm slow and impractical. 
# example: 
# minima = mm.Minima(fes, precise=False, nbins=8)
# 
minima = mm.Minima(fes, precise=True, temp=300.0, energy_unit="kJ/mol")

In [ ]:
# plot the free energy surface with minima
minima.plot(contours_spacing=20)

In [ ]:
# print the list of local minima
print(minima.minima)

In [ ]:
# construct free energy profile
prof = mm.FEProfile(minima,hills)

# plot the free energy profile
prof.plot(legend=True, png_name="fep.png")

In [ ]:
# You can also remove one CV from existing free energy surface: 
fes_CV1 = fes.removeCV(CV=2)
# You can work with the new FES object like with a new FES: 
minima_1 = mm.Minima(fes_CV1)
minima_1.minima

In [ ]:
# ... and plot it: 
minima_1.plot()

In [ ]:
# You can visualize 2D FES as surface plot: 
fes.surface_plot()

In [ ]:
# If you have 3D FES, it can be easier to visualise the FES as an animation 
# showing different isosurfaces with different free energy levels during time:
# 
fes.make_gif(gif_name="fes_animation")

In [ ]:
<img src="fes_animation.gif" width="750" align="center"/>

In [ ]:
# It is also possible to make animation of the flooding of FES during the simulation
fes.flooding_animation(step=1000, gif_name = "flooding.gif", fps=10, enable_loop=True, contours_spacing=10, with_minima=True, use_vmax_from_end=False)

<img src="flooding.gif" width="750" align="center"/>